In [1]:
import pandas as pd
df_train = pd.read_csv('df_jan_train.csv', engine='python')

In [2]:
sat_id = 123

In [3]:
train = df_train[df_train.sat_id==sat_id].copy()
train['epoch'] = pd.to_datetime(train['epoch'], infer_datetime_format=True)
train['epoch'] = train['epoch'].dt.strftime('%Y-%m-%d %H:%M:%S')
train.head(2)

sat_id                epoch        date  day  hour  minute  \
38111     123  2014-01-01 00:00:00  2014-01-01    1     0       0   
38112     123  2014-01-01 00:29:40  2014-01-01    1     0      29   

                  x             y            z               next_epoch  \
38111  14511.024670  13023.085484 -3929.568361  2014-01-01 00:29:40.162   
38112  19297.897633   6014.609551 -1580.558834  2014-01-01 00:59:20.323   

       time_diff        next_x       next_y       next_z  
38111  29.669367  19297.897633  6014.609551 -1580.558834  
38112  29.669350  21225.256604 -1908.309300  1010.219760

In [4]:
cols = ['epoch','x','y','z','time_diff','next_x','next_y','next_z'] #'time_diff'
train = train[cols]
train.head(2)

epoch             x             y            z  \
38111  2014-01-01 00:00:00  14511.024670  13023.085484 -3929.568361   
38112  2014-01-01 00:29:40  19297.897633   6014.609551 -1580.558834   

       time_diff        next_x       next_y       next_z  
38111  29.669367  19297.897633  6014.609551 -1580.558834  
38112  29.669350  21225.256604 -1908.309300  1010.219760

In [5]:
from pyFTS.partitioners import Grid, Util as pUtil
from pyFTS.common import Transformations
from pyFTS.models.multivariate import common, variable, mvfts
from pyFTS.models.seasonal import partitioner as seasonal
from pyFTS.models.seasonal.common import DateTime

In [6]:
print(min(train.epoch),max(train.epoch))
print([str(i) for i in list(range(1,25))])

2014-01-01 00:00:00 2014-01-24 23:05:28
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [7]:
names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']

In [8]:
import matplotlib.pylab as plt
import matplotlib.ticker as mticker


def get_model(df, target_col):
    bc = Transformations.BoxCox(0)
    diff = Transformations.Differential(1)

    np = 10

    model = mvfts.MVFTS()

    fig, axes = plt.subplots(nrows=7, ncols=1,figsize=[15,10])

    sp = {'seasonality': DateTime.day_of_month , 'names': names}
    v_day = variable.Variable("Day", data_label="epoch", partitioner=seasonal.TimeGridPartitioner, npart=len(names), data=df, partitioner_specific=sp)
    model.append_variable(v_day)
    v_day.partitioner.plot(axes[0])

    sp = {'seasonality': DateTime.hour_of_day}
    v_hour = variable.Variable("Hour", data_label="epoch", partitioner=seasonal.TimeGridPartitioner, npart=24, data=df, partitioner_specific=sp)
    model.append_variable(v_hour)
    v_hour.partitioner.plot(axes[1])

    v_x = variable.Variable("x", data_label="x", partitioner=Grid.GridPartitioner, npart=np, data=df)
    model.append_variable(v_x)
    v_x.partitioner.plot(axes[2])

    v_y = variable.Variable("y", data_label="y", partitioner=Grid.GridPartitioner, npart=np, data=df)
    model.append_variable(v_y)
    v_y.partitioner.plot(axes[3])

    v_z = variable.Variable("z", data_label="z", partitioner=Grid.GridPartitioner, npart=np, data=df)
    model.append_variable(v_z)
    v_z.partitioner.plot(axes[4])
    
    v_time_diff = variable.Variable("Time diff", data_label="time_diff", partitioner=Grid.GridPartitioner, npart=np, data=df)
    model.append_variable(v_time_diff)
    v_time_diff.partitioner.plot(axes[5])    

    v_next = variable.Variable(target_col, data_label=target_col, partitioner=Grid.GridPartitioner, npart=20, data=df)
    model.append_variable(v_next)
    v_next.partitioner.plot(axes[6])

    model.target_variable = v_next

    plt.tight_layout()

    return model

In [19]:
dataset.loc[:,'epoch'] = dataset['epoch'].astype('datetime64[ns]')

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [20]:
from pyFTS.models import hofts, pwfts

dataset = train[train.columns.difference(['next_y','next_z'])]

partitioner = Grid.GridPartitioner(data=dataset, npart=30)
partitioner_diff = Grid.GridPartitioner(data=dataset, npart=10, transformation=tdiff)


pfts1_taiex = pwfts.ProbabilisticWeightedFTS("1", partitioner=partitioner)
pfts1_taiex.fit(dataset, save_model=True, file_path='pwfts')

TypeError: '<=' not supported between instances of 'str' and 'float'

In [10]:
model_x.predict(train[['epoch','x','y','z','time_diff']], type='distribution')

NotImplementedError: This model do not perform one step ahead distribution forecasts!

In [ ]:
model_x = get_model(train[train.columns.difference(['next_y','next_z'])], "next_x")
model_x.fit(train[['epoch','x','y','z','time_diff','next_x']], num_batches=200, save=True, batch_save=True, file_path='x', 
          distributed=True, nodes=['192.168.0.110','192.168.0.106'], 
          batch_save_interval=10)

In [ ]:
model_y = get_model(train[train.columns.difference(['next_x','next_z'])], "next_y")
model_y.fit(train[['epoch','x','y','z','time_diff','next_y']], num_batches=200, save=True, batch_save=True, file_path='y', 
          distributed=True, nodes=['192.168.0.110','192.168.0.106'], 
          batch_save_interval=10)

In [ ]:
model_z = get_model(train[train.columns.difference(['next_x','next_y'])], "next_z")
model_z.fit(train[['epoch','x','y','z','time_diff','next_z']], num_batches=200, save=True, batch_save=True, file_path='z',
          distributed=True, nodes=['192.168.0.110','192.168.0.106'], 
          batch_save_interval=10)

In [ ]:
from datetime import datetime
#_range = train['epoch'].values
print(datetime.now())
#forecasted_x = model_x.predict(train[['epoch','x','y','z','time_diff','next_x']], type='point', steps_ahead=1)
#forecasted_y = model_y.predict(train[['epoch','x','y','z','time_diff','next_y']], type='point', steps_ahead=1)
#forecasted_z = model_z.predict(train[['epoch','x','y','z','time_diff','next_z']], type='point', steps_ahead=1)

forecasted_x = model_x.forecast(train[['epoch','x','y','z','time_diff','next_x']], distributed=True, nodes=['192.168.0.110'])
forecasted_y = model_y.forecast(train[['epoch','x','y','z','time_diff','next_y']], distributed=True, nodes=['192.168.0.110'])
forecasted_z = model_z.forecast(train[['epoch','x','y','z','time_diff','next_z']], distributed=True, nodes=['192.168.0.110'])

print(datetime.now())

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=[30,10])
#axs[0].plot(_range,train['next_x'].values)
#axs[1].plot(_range,forecasted_x)
axs[0].plot(train['next_x'].values)
axs[1].plot(forecasted_x)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=[30,10])
#axs[0].plot(_range,train['next_y'].values)
#axs[1].plot(_range,forecasted_y)
axs[0].plot(train['next_y'].values)
axs[1].plot(forecasted_y)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=[30,10])
#axs[0].plot(_range,train['next_z'].values)
#axs[1].plot(_range,forecasted_z)
axs[0].plot(train['next_z'].values)
axs[1].plot(forecasted_z)

In [ ]:
import numpy as np

y_true = train[['next_x','next_y','next_z']].values
y_pred = np.array([forecasted_x,forecasted_y,forecasted_z]).T

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_true, y_pred , squared=False)
rmse

In [ ]:
from datetime import datetime
print(datetime.now())
#forecasted_x_aux = model_x.predict(train[['epoch','x','y','z','time_diff','next_x']])
#forecasted_y_aux = model_y.predict(train[['epoch','x','y','z','time_diff','next_y']])
#forecasted_z_aux = model_z.predict(train[['epoch','x','y','z','time_diff','next_z']])
forecasted_x_aux = model_x.forecast(train[['epoch','x','y','z','time_diff','next_x']])
forecasted_y_aux = model_y.forecast(train[['epoch','x','y','z','time_diff','next_y']])
forecasted_z_aux = model_z.forecast(train[['epoch','x','y','z','time_diff','next_z']])
print(datetime.now())

In [ ]:
pd.DataFrame(np.array([forecasted_x_aux,forecasted_y_aux,forecasted_z_aux]).T, columns=['x','y','z']).head()

In [ ]:
train.head()

In [ ]:
import math
aux = train.iloc[:1,:].copy()
n = math.floor(aux.time_diff.max())
aux = pd.DataFrame(np.repeat(aux.values, n, axis=0))
aux.columns = train.columns
for i in range(0,n):
    aux.loc[i,'time_diff'] = aux.loc[i,'time_diff'] + (i-n+1)
    #aux.loc[i,'time_diff'] = math.floor(aux.loc[i,'time_diff'] + (i-n+1))
aux.head()

#from datetime import timedelta
#aux['epoch'] = aux['epoch'].astype('datetime64[ns]')
#aux['epoch'] = aux.apply(lambda x: x.epoch + timedelta(minutes=x.time_diff-1), axis=1)
#aux.head()

#aux['rand'] = np.random.randint(0,aux['x'].max(),(len(aux), 1))
#aux['x'] = aux['x'] + aux['rand']
#aux.drop(columns=['rand'], inplace=True)
aux = aux[['epoch','x','y','z','time_diff']]
aux.head()

In [ ]:
print(datetime.now())
forecasted_x_aux = model_x.forecast(train[['epoch','x','y','z','time_diff']])
forecasted_y_aux = model_y.forecast(train[['epoch','x','y','z','time_diff']])
forecasted_z_aux = model_z.forecast(train[['epoch','x','y','z','time_diff']])
print(datetime.now())

In [21]:
model_x.forecast??

Signature: model_x.forecast(data, **kwargs)
Docstring:
Point forecast one step ahead

:param data: time series data with the minimal length equal to the max_lag of the model
:param kwargs: model specific parameters
:return: a list with the forecasted values
Source:   
    def forecast(self, data, **kwargs):
        ret = []
        ndata = self.apply_transformations(data)
        c = 0
        for index, row in ndata.iterrows() if isinstance(ndata, pd.DataFrame) else enumerate(ndata):
            data_point = self.format_data(row)
            flrs = self.generate_lhs_flrs(data_point)
            mvs = []
            mps = []
            for flr in flrs:
                flrg = mvflrg.FLRG(lhs=flr.LHS)
                if flrg.get_key() not in self.flrgs:
                    #Naïve approach is applied when no rules were found
                    if self.target_variable.name in flrg.LHS:
                        fs = flrg.LHS[self.target_variable.name]
                        fset = self.ta